<a href="https://colab.research.google.com/github/EchenimEdwin/https-github.com-users-EchenimEdwin-/blob/main/SCADA_System_with_AI_ML_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import random
from collections import deque # For storing recent data

# --- Configuration Parameters ---
# Sensor data limits
TEMP_MIN, TEMP_MAX = 20.0, 30.0
PRESSURE_MIN, PRESSURE_MAX = 50.0, 70.0
FLOW_MIN, FLOW_MAX = 100.0, 150.0

# Alarm thresholds
TEMP_ALARM_HIGH = 28.0
PRESSURE_ALARM_HIGH = 65.0
FLOW_ALARM_LOW = 110.0

# Data history for AI/ML (e.g., last 10 readings)
HISTORY_LENGTH = 10

# --- Global Data Storage ---
sensor_data_history = {
    'temperature': deque(maxlen=HISTORY_LENGTH),
    'pressure': deque(maxlen=HISTORY_LENGTH),
    'flow': deque(maxlen=HISTORY_LENGTH),
}
current_alarms = set() # Stores active alarm messages

# --- 1. Data Generation Function ---
def generate_sensor_data():
    """Simulates real-time sensor data with some fluctuations."""
    temp = round(random.uniform(TEMP_MIN, TEMP_MAX) + random.gauss(0, 1), 2)
    pressure = round(random.uniform(PRESSURE_MIN, PRESSURE_MAX) + random.gauss(0, 0.5), 2)
    flow = round(random.uniform(FLOW_MIN, FLOW_MAX) + random.gauss(0, 2), 2)

    # Introduce occasional "faults" for testing alarm/AI
    if random.random() < 0.1: # 10% chance of a fault
        fault_type = random.choice(['temp_spike', 'pressure_drop', 'flow_surge'])
        if fault_type == 'temp_spike':
            temp = round(random.uniform(TEMP_ALARM_HIGH + 1, TEMP_MAX + 5), 2)
        elif fault_type == 'pressure_drop':
            pressure = round(random.uniform(PRESSURE_MIN - 5, PRESSURE_ALARM_HIGH - 1), 2)
        elif fault_type == 'flow_surge':
            flow = round(random.uniform(FLOW_MIN - 10, FLOW_ALARM_LOW - 1), 2)

    return {'temperature': temp, 'pressure': pressure, 'flow': flow}

# --- 2. Data Visualization Function (Text-based SCADA Dashboard) ---
def display_dashboard(data, alarms, ai_insight):
    """
    Prints a simple text-based SCADA dashboard.
    Clears the console for a 'real-time' effect.
    """
    # For a real application, you'd use a GUI library like Tkinter, PyQt, or a web framework.
    # This simulates clearing the screen in a terminal.
    # print("\033c", end="") # Clear console - might not work on all terminals

    print("-" * 50)
    print(f"SCADA SYSTEM DASHBOARD - {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print("-" * 50)

    print("\n--- Sensor Readings ---")
    for sensor, value in data.items():
        print(f"  {sensor.capitalize():<12}: {value:>8.2f}")

    print("\n--- Active Alarms ---")
    if alarms:
        for alarm in sorted(list(alarms)):
            print(f"  [!!!] {alarm}")
    else:
        print("  No active alarms.")

    print("\n--- AI/ML Insight ---")
    print(f"  {ai_insight}")

    print("-" * 50)
    print("\n") # Add some space before the next update

# --- 3. Alarm Triggering Function ---
def check_alarms(data):
    """
    Checks sensor data against predefined thresholds and updates active alarms.
    """
    new_alarms = set()

    if data['temperature'] > TEMP_ALARM_HIGH:
        new_alarms.add(f"High Temperature: {data['temperature']:.2f}°C (Threshold: >{TEMP_ALARM_HIGH}°C)")
    else:
        current_alarms.discard(f"High Temperature: {data['temperature']:.2f}°C (Threshold: >{TEMP_ALARM_HIGH}°C)") # Remove if cleared

    if data['pressure'] > PRESSURE_ALARM_HIGH:
        new_alarms.add(f"High Pressure: {data['pressure']:.2f} PSI (Threshold: >{PRESSURE_ALARM_HIGH} PSI)")
    else:
        current_alarms.discard(f"High Pressure: {data['pressure']:.2f} PSI (Threshold: >{PRESSURE_ALARM_HIGH} PSI)")

    if data['flow'] < FLOW_ALARM_LOW:
        new_alarms.add(f"Low Flow Rate: {data['flow']:.2f} L/s (Threshold: <{FLOW_ALARM_LOW} L/s)")
    else:
        current_alarms.discard(f"Low Flow Rate: {data['flow']:.2f} L/s (Threshold: <{FLOW_ALARM_LOW} L/s)")

    # Update global alarms set
    current_alarms.update(new_alarms)

# --- 4. AI/ML Model Integration (Simulated) ---
def analyze_with_ai_ml(data_history):
    """
    Simulates an AI/ML model analyzing historical and current data.
    In a real scenario, this would involve trained models (e.g., TensorFlow, PyTorch).
    For this example, it's a simple rule-based anomaly detection.
    """
    latest_temp = data_history['temperature'][-1] if data_history['temperature'] else None
    latest_pressure = data_history['pressure'][-1] if data_history['pressure'] else None
    latest_flow = data_history['flow'][-1] if data_history['flow'] else None

    # Simple anomaly detection: check for sudden large changes
    insight = "System operating normally."

    if len(data_history['temperature']) >= 2:
        temp_diff = latest_temp - data_history['temperature'][-2]
        if abs(temp_diff) > 3.0: # Significant change
            insight = "AI Alert: Unusual temperature fluctuation detected!"
            if temp_diff > 0: insight += f" (Rising by {temp_diff:.2f}°C)"
            else: insight += f" (Falling by {abs(temp_diff):.2f}°C)"
            return insight

    if len(data_history['pressure']) >= 2:
        pressure_diff = latest_pressure - data_history['pressure'][-2]
        if abs(pressure_diff) > 2.0:
            insight = "AI Alert: Sudden pressure change detected!"
            if pressure_diff > 0: insight += f" (Rising by {pressure_diff:.2f} PSI)"
            else: insight += f" (Falling by {abs(pressure_diff):.2f} PSI)"
            return insight

    if len(data_history['flow']) >= 2:
        flow_diff = latest_flow - data_history['flow'][-2]
        if abs(flow_diff) > 5.0:
            insight = "AI Alert: Significant flow rate deviation detected!"
            if flow_diff > 0: insight += f" (Rising by {flow_diff:.2f} L/s)"
            else: insight += f" (Falling by {abs(flow_diff):.2f} L/s)"
            return insight

    # Check for sustained out-of-norm behavior not necessarily an 'alarm' but potentially problematic
    if all(t > TEMP_ALARM_HIGH - 1 for t in list(data_history['temperature'])) and len(data_history['temperature']) == HISTORY_LENGTH:
        insight = "AI Suggestion: Sustained high temperature readings, consider inspection."
        return insight

    if all(f < FLOW_ALARM_LOW + 5 for f in list(data_history['flow'])) and len(data_history['flow']) == HISTORY_LENGTH:
        insight = "AI Suggestion: Flow rate consistently low, investigate potential blockage."
        return insight


    return insight


# --- Main SCADA Loop ---
def run_scada_system(duration_seconds=60, update_interval_seconds=2):
    """
    Runs the simulated SCADA system for a specified duration.
    """
    print("Starting SCADA System Simulation...")
    start_time = time.time()

    while (time.time() - start_time) < duration_seconds:
        # 1. Generate new data
        new_data = generate_sensor_data()

        # Update data history for AI/ML
        for sensor, value in new_data.items():
            sensor_data_history[sensor].append(value)

        # 2. Check for alarms
        check_alarms(new_data)

        # 3. Get AI/ML insight (only if enough data is available)
        ai_insight = "Waiting for sufficient data for AI analysis..."
        if all(len(history) == HISTORY_LENGTH for history in sensor_data_history.values()):
            ai_insight = analyze_with_ai_ml(sensor_data_history)

        # 4. Display dashboard
        display_dashboard(new_data, current_alarms, ai_insight)

        time.sleep(update_interval_seconds)

    print("\nSCADA System Simulation Finished.")
    print("Final Active Alarms:", current_alarms if current_alarms else "None")


if __name__ == "__main__":
    # Run the simulation for 120 seconds, updating every 2 seconds
    run_scada_system(duration_seconds=120, update_interval_seconds=2)

Starting SCADA System Simulation...
--------------------------------------------------
SCADA SYSTEM DASHBOARD - 2025-06-21 23:38:36
--------------------------------------------------

--- Sensor Readings ---
  Temperature :    22.29
  Pressure    :    67.83
  Flow        :    96.65

--- Active Alarms ---
  [!!!] High Pressure: 67.83 PSI (Threshold: >65.0 PSI)
  [!!!] Low Flow Rate: 96.65 L/s (Threshold: <110.0 L/s)

--- AI/ML Insight ---
  Waiting for sufficient data for AI analysis...
--------------------------------------------------


--------------------------------------------------
SCADA SYSTEM DASHBOARD - 2025-06-21 23:38:38
--------------------------------------------------

--- Sensor Readings ---
  Temperature :    21.16
  Pressure    :    62.41
  Flow        :   136.13

--- Active Alarms ---
  [!!!] High Pressure: 67.83 PSI (Threshold: >65.0 PSI)
  [!!!] Low Flow Rate: 96.65 L/s (Threshold: <110.0 L/s)

--- AI/ML Insight ---
  Waiting for sufficient data for AI analysis...
-